# Limits
First, we establish a rate limiter for Binance. This will avoid overcharging the rate limiter and potentially hitting the rate limit that can ban your account.

In [1]:
from panzer.limits import BinanceRateLimiter

In [2]:
l = BinanceRateLimiter()

Rate limits: [{'rateLimitType': 'REQUEST_WEIGHT', 'interval': 'MINUTE', 'intervalNum': 1, 'limit': 6000}, {'rateLimitType': 'ORDERS', 'interval': 'SECOND', 'intervalNum': 10, 'limit': 100}, {'rateLimitType': 'ORDERS', 'interval': 'DAY', 'intervalNum': 1, 'limit': 200000}, {'rateLimitType': 'RAW_REQUESTS', 'interval': 'MINUTE', 'intervalNum': 5, 'limit': 61000}]


2024-09-18	 20:00:11     INFO Updated server-local time offset: 113 ms


In [3]:
l.get()

{'orders_limit_per_ten_seconds': 100,
 'orders_limit_per_day': 200000,
 'weight_limit_per_minute': 6000,
 'raw_limit_per_5_minutes': 61000,
 'server_time_offset': 113,
 'five_minutes_counts': {5755608: 2},
 'minutes_weights': {28778040: 21},
 'ten_seconds_orders_counts': {},
 'daily_orders_count': {}}

# Credentials
First, we add api keys and secret to the credential manager. If not, they will be asked by prompt. Adding with is_sensitive parameter will save the credentials securely. If not 
provided then it will be automatically secured if name of the credential is "api_key" or "api_secret".

In [4]:
from panzer.keys import CredentialManager

In [5]:
c = CredentialManager()

In [6]:
c.credentials

{}

## Cached credentials
If CredentialManager is asked for a credential that is already in the cache, it will return the cached value instead of asking for it again. Even it is not loaded in the object 
at first creation, it will add from disk cache automatically when needed, and it will prompt for it if it is not loaded. 

In [7]:
c.add("api_key", "test", is_sensitive=True, overwrite=False)
c.add("api_secret", "test", is_sensitive=True, overwrite=False)

2024-09-18	 20:00:13     INFO Variable api_key exists in cache file. To renew it, pass overwrite parameter to True or delete line in cache file at: C:\Users\hanca\.panzer_creds
2024-09-18	 20:00:13     INFO Variable api_secret exists in cache file. To renew it, pass overwrite parameter to True or delete line in cache file at: C:\Users\hanca\.panzer_creds


'uU7DyqKUkvOX4OsMpeBetxy05s2ZGz6CY0DgkOHhlvrbakh/fLU7K4MXY3IHBJbqs+qlznAVCmGR1fWzprNR42A1YsUpRK8B3m3xBFPZ40g='

In [8]:
c.get("api_secret", decrypt=True)

'test'

In [9]:
# renew to load real creds from file
c = CredentialManager()

In [10]:
# cached in file can be retrieved with `file_manager` attribute.
# c.file_manager

In [11]:
c

{}

# Examples of API calls
Credentials for API calls are going to be asked by prompt the first time if they are not in the creds cache.


In [12]:
from panzer.request import get, post

## Get klines example
Assuming credentials are already in the cache, will check limits and then make the API call.

In [13]:
url = 'https://api.binance.com/api/v3/klines'
weight=2
params = {
    "symbol": "BTCUSDT",  # Par BTCUSDT
    "interval": "1m",     # Intervalo de 1 minuto
    "limit": 3            # Limitar a las últimas 5 velas
}

In [14]:
l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False)

True

In [15]:
response, headers = get(params=params, 
                        url=url,
                        full_sign=False,
                        server_time_offset=l.server_time_offset,)

In [16]:
# first limits update will notify sync of the object with api server if they differ
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [17]:
response

[[1726682280000,
  '60022.21000000',
  '60048.00000000',
  '59980.00000000',
  '59980.00000000',
  '34.39332000',
  1726682339999,
  '2064263.49788780',
  3447,
  '11.57473000',
  '694749.56589450',
  '0'],
 [1726682340000,
  '59980.00000000',
  '60025.43000000',
  '59980.00000000',
  '60013.01000000',
  '54.24845000',
  1726682399999,
  '3254830.60990270',
  4773,
  '35.85856000',
  '2151365.38717100',
  '0'],
 [1726682400000,
  '60013.02000000',
  '60486.59000000',
  '59759.84000000',
  '60432.17000000',
  '475.56764000',
  1726682459999,
  '28610493.33113570',
  22127,
  '249.78255000',
  '15030836.30945870',
  '0']]

In [18]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '226',
 'Connection': 'keep-alive',
 'Date': 'Wed, 18 Sep 2024 18:00:15 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': '8a8b3852-8280-48ef-88bb-d59cb1528557',
 'x-mbx-used-weight': '23',
 'x-mbx-used-weight-1m': '23',
 'content-encoding': 'gzip',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 a2f1af60340347fb7ac41d41acebdd2c.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'CDG55-P1',
 'X-Amz-Cf-Id': 'WRKD6td9iuOy6TyvRlDjL0

### Without weight
It can get weight for url from a local database created upon use of the module. Also is_order can be inferred from the url.

In [19]:
l.can_make_request(url=url, params_qty=len(params), weight=None, is_order=None)
response, headers = get(params=params, 
                        url=url,
                        full_sign=False,
                        server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

## Test order

In [20]:
url = 'https://api.binance.com/api/v3/order/test'
weight = 1

# timestamp is automatically added when signed call
params = {'symbol': "BTCUSDT",
          'side': "SELL",
          'type': "LIMIT",
          'timeInForce': 'GTC',
          'quantity': 0.001,
          'price': 80000,
          'recvWindow': 10000}

In [21]:
if l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False):
    response, headers = post(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [22]:
response

{}

In [23]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '2',
 'Connection': 'keep-alive',
 'Date': 'Wed, 18 Sep 2024 18:00:16 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': '2da82bbd-7ee9-490e-9b5a-e0c19a259cfc',
 'x-mbx-used-weight': '26',
 'x-mbx-used-weight-1m': '26',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 614e1f0bd8649d5a99ab4e4cdbdaddb0.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'CDG55-P1',
 'X-Amz-Cf-Id': 'rawoc-00DLfZL21Z7zzE3oxt41D1sRzQIwaqFz8S5tpQ3fjQUcQFX

## Trades

In [24]:
url = 'https://api.binance.com/api/v3/myTrades'
weight = 20
params = {
    'symbol': 'BTCUSDT',                   # The trading pair
    'limit': 3,                            # Optional: Limit the number of trades to retrieve (default 500)
    'recvWindow': 5000                     # Optional: Time window for the request (default 5000 ms)
}

In [25]:
if l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False):
    response, headers = get(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [26]:
response

[{'symbol': 'BTCUSDT',
  'id': 3315169490,
  'orderId': 23619145064,
  'orderListId': -1,
  'price': '41000.00000000',
  'qty': '0.00175000',
  'quoteQty': '71.75000000',
  'commission': '0.00023627',
  'commissionAsset': 'BNB',
  'time': 1702260789687,
  'isBuyer': True,
  'isMaker': True,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 3316660873,
  'orderId': 23687231478,
  'orderListId': -1,
  'price': '40986.19000000',
  'qty': '0.00128000',
  'quoteQty': '52.46232320',
  'commission': '0.00016332',
  'commissionAsset': 'BNB',
  'time': 1702320683140,
  'isBuyer': True,
  'isMaker': True,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 3327971249,
  'orderId': 23818929439,
  'orderListId': -1,
  'price': '43012.60000000',
  'qty': '0.00303000',
  'quoteQty': '130.32817800',
  'commission': '0.00039912',
  'commissionAsset': 'BNB',
  'time': 1702981648395,
  'isBuyer': False,
  'isMaker': False,
  'isBestMatch': True}]

In [27]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '311',
 'Connection': 'keep-alive',
 'Date': 'Wed, 18 Sep 2024 18:00:16 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': 'ba0df13b-ff39-43bd-b7ed-c44b752373c7',
 'x-mbx-used-weight': '46',
 'x-mbx-used-weight-1m': '46',
 'content-encoding': 'gzip',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 da8df4277339b60b1b379ebb4c80b78e.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'CDG55-P1',
 'X-Amz-Cf-Id': 'CE_PAK4CeMJKXbYPbgEVXo

In [28]:
l.get()

{'orders_limit_per_ten_seconds': 100,
 'orders_limit_per_day': 200000,
 'weight_limit_per_minute': 6000,
 'raw_limit_per_5_minutes': 61000,
 'server_time_offset': 113,
 'five_minutes_counts': {5755608: 6},
 'minutes_weights': {28778040: 46},
 'ten_seconds_orders_counts': {},
 'daily_orders_count': {}}